# Pre trained network
![imagen](https://www.researchgate.net/publication/336874848/figure/fig1/AS:819325225144320@1572353764073/Illustrations-of-transfer-learning-a-neural-network-is-pretrained-on-ImageNet-and.png)

Estas son las arquitecturas de redes neuronales más utilizadas en la comunidad. Para más detalle sobre el funcionamiento de cada red, consultar el *Hands on Machine Learning for Python*.
* VGG-16
* VGG-19
* Inception V3
* XCeption
* ResNet-50

Estas redes las podemos incorporar entrenadas, o sin entrenar.

## ResNet50V2

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2

from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

In [5]:
#### CODE ####
base_model = ResNet50V2(input_shape=(224,224,3),  #La red ResNet50V2, está abierta pero 'sólo' acepta entrada de 224 * 224 pixeles
                        include_top=True, #Incluye todo, no solo la parte de Convoluciones si no con sus propias ce NN(redNeuronal) también.
                        weights= 'imagenet', #La red de imgenes con sus categorias
                        classifier_activation= 'softmax')

102875136/102869336 [==============================] - 492s 5us/step


In [8]:
#### CODE ####
base_model.summary()

elu[0][0]        
__________________________________________________________________________________________________
conv4_block1_out (Add)          (None, 14, 14, 1024) 0           conv4_block1_0_conv[0][0]        
                                                                 conv4_block1_3_conv[0][0]        
__________________________________________________________________________________________________
conv4_block2_preact_bn (BatchNo (None, 14, 14, 1024) 4096        conv4_block1_out[0][0]           
__________________________________________________________________________________________________
conv4_block2_preact_relu (Activ (None, 14, 14, 1024) 0           conv4_block2_preact_bn[0][0]     
__________________________________________________________________________________________________
conv4_block2_1_conv (Conv2D)    (None, 14, 14, 256)  262144      conv4_block2_preact_relu[0][0]   
___________________________________________________________________________________________

Cargamos algunas imagenes desde local, para ver qué tal funciona la red ResNet50V2 ya entrenada.

In [9]:
import os


def read_data(path):
    X = []

    for file in os.listdir(path):
        image = imread(path + '/' + file)
        smallimage = cv2.resize(image, (224, 224))
        print(path + '/' + file)
        
        X.append(smallimage)

    return np.array(X)
    

x_test = read_data('img')

# Procesar las imagenes tal y como entran en el modelo
x_test = preprocess_input(x_test)
print(x_test.shape)

img/bear-1.jpg
img/cat.8016.jpg
img/cat.8037.jpg
img/dog.11856.jpg
img/dog.11857.jpg
img/horse.jpg
img/karate.jpg
img/pizza.jpg
(8, 224, 224, 3)


In [10]:
#### CODE ####
preds = base_model.predict(x_test)

#decode the results into a list of tuples (class, description, probability)
decodes = decode_predictions(preds, top= 5)

for j in decodes:
    print("#########")
    for i, decode in enumerate(j):
        print("Predicted:\n", decode[1], decode[2])

40960/35363 [==================================] - 1s 16us/step
#########
Predicted:
 brown_bear 0.99944514
Predicted:
 chow 0.00054139504
Predicted:
 American_black_bear 8.784533e-06
Predicted:
 ice_bear 1.1080934e-06
Predicted:
 howler_monkey 4.4057498e-07
#########
Predicted:
 Egyptian_cat 0.659723
Predicted:
 Siamese_cat 0.15741089
Predicted:
 tiger_cat 0.06260011
Predicted:
 lynx 0.022836583
Predicted:
 tabby 0.018356169
#########
Predicted:
 Egyptian_cat 0.8211865
Predicted:
 lynx 0.12932149
Predicted:
 Siamese_cat 0.021827206
Predicted:
 tabby 0.018324552
Predicted:
 tiger_cat 0.006797261
#########
Predicted:
 Rottweiler 0.76189536
Predicted:
 Brabancon_griffon 0.0746136
Predicted:
 Staffordshire_bullterrier 0.028572641
Predicted:
 EntleBucher 0.019904258
Predicted:
 Doberman 0.019285118
#########
Predicted:
 collie 0.99525565
Predicted:
 Shetland_sheepdog 0.004744308
Predicted:
 yellow_lady's_slipper 9.751216e-09
Predicted:
 goldfinch 8.175372e-09
Predicted:
 groenendael 7.4755

## VGG16
En este caso vamos a importar la red VGG16, que utilizaremos como red preentrenada y completaremos con una fully connected layer. 

In [14]:
path= "../../../../DataSets/dogs_cats/"
folder_train = "train/"
folder_test = "test1/"

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

IM_SIZE=64

TRAIN_PATH = path + folder_train

filenames = os.listdir(TRAIN_PATH)
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    categories.append(category)
    
df = pd.DataFrame({
    'filenames': filenames,
    'category': categories
})

train_df, validate_df = train_test_split(df,
                                         test_size=0.20,
                                         random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [18]:
os.listdir(TRAIN_PATH)[-10:]

['dog.9990.jpg',
 'dog.9991.jpg',
 'dog.9992.jpg',
 'dog.9993.jpg',
 'dog.9994.jpg',
 'dog.9995.jpg',
 'dog.9996.jpg',
 'dog.9997.jpg',
 'dog.9998.jpg',
 'dog.9999.jpg']

In [19]:
from keras.preprocessing.image import ImageDataGenerator


# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale = 1.0/255. ) #Sólo escala los registros

In [20]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [21]:
#### CODE ####
from tensorflow.keras.applications.vgg16 import VGG16 

base_model = VGG16(input_shape=(IM_SIZE, IM_SIZE, 3),
                   include_top= False, #No me la conectes a la última red_nn (NN==> NeuronalNet)
                   weights= 'imagenet') #los pesos los saque de una base de datos 'image_net' de imagenes_clasificadas

#No queremos que entrene en el fit
for layer in base_model.layers:
    layer.trainable = False
    
##### FULLY CONNECTED LAYER #####
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [22]:
vgghist = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 10)

Epoch 1/10
1000/1000 [==============================] - 554s 553ms/step - loss: 0.6092 - acc: 0.6843 - val_loss: 0.4660 - val_acc: 0.7738
Epoch 2/10
1000/1000 [==============================] - 554s 554ms/step - loss: 0.5249 - acc: 0.7332 - val_loss: 0.4574 - val_acc: 0.7826
Epoch 3/10
1000/1000 [==============================] - 550s 550ms/step - loss: 0.5020 - acc: 0.7536 - val_loss: 0.4500 - val_acc: 0.7810
Epoch 4/10
1000/1000 [==============================] - 516s 516ms/step - loss: 0.4962 - acc: 0.7551 - val_loss: 0.4392 - val_acc: 0.7906
Epoch 5/10
1000/1000 [==============================] - 532s 532ms/step - loss: 0.4836 - acc: 0.7632 - val_loss: 0.4356 - val_acc: 0.7894
Epoch 6/10
1000/1000 [==============================] - 530s 529ms/step - loss: 0.4885 - acc: 0.7550 - val_loss: 0.4273 - val_acc: 0.7984
Epoch 7/10
1000/1000 [==============================] - 530s 530ms/step - loss: 0.4863 - acc: 0.7593 - val_loss: 0.4377 - val_acc: 0.7890
Epoch 8/10
1000/1000 [============

### Inception

In [23]:
#Otra plantilla

In [ ]:
IM_SIZE = 150

#flow training images in batches of 20 using train datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                      TRAIN_PATH,
                                                      x_col = 'filename',
                                                      y_col = 'category',
                                                      batch_size = 20,
                                                      class_mode = 'binary',
                                                      target_size = (IM_SIZE, IM_SIZE))

#flow validation images in batches of 20 using test datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col = 'filename',
                                                              y_col = 'category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape = (IM_SIZE, IM_SIZE, 3),
                                        include_top = False,
                                        weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(base_model.output)

x = layers.Dense(units= 1024, activation = 'relu')(x)
x = layers.Dropout(0.2)(x)

x = layers.Dense(units= 1, activation = 'sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

In [ ]:
model.compile(optimizer = RMSprop(lr = 0.0001),
              loss = 'binary_crossentropy',
              metrics =  ['acc'])

inc_history = model.fit(train_generator,
                        validation_data = validation_generator,
                        steps_per_epoch = 100,
                        epochs = 10)

In [ ]:
### sintaxis ### 

"""
Note that there are 2 ways of writing the same code:

model = VGG19(weigths= 'imagenet', include_top = False)
model.trainable = False

layer1 = Flatten(name= 'flat') (model)
layer2 = Dense(units= 512, activation = 'relu', name = 'fc1')(layer1)
layer3 = Dense(units= 512, activation = 'relu', name = 'fc2')(layer2)
layer4 = Dense(units= 10, activation = 'softmax', name = 'predictions')(layer3)

#That can be rewritten as:

model = VGG19(weigths= 'imagenet', include_top = False)
model.trainable = False

model.add(Flatten(name= 'flat))
model.add(Dense(units= 512, activation = 'relu', name = 'fc1'))
model.add(Dense(units= 512, activation = 'relu', name = 'fc2'))
model.add(Dense(units= 10, activation = 'softmax', name = 'predictions'))
"""

## ResNet50V2 sin entrenar

In [ ]:
IM_SIZE = 32

base_model = ResNet50V2(input_shape= (IM_SIZE, IM_SIZE, 3),
                        include_top = False,
                        classifier_activation = 'softmax')

In [24]:
#Flow training images in batches of 20 using train_datagen generator

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col = 'filenames',
                                                    y_col = 'category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))


#Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col = 'filename',
                                                              y_col = 'category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))


Found 20000 validated image filenames belonging to 2 classes.


NameError: name 'IM' is not defined

In [ ]:
#Flatten the output layer to 1 dimension
x = layers.Flatten(base_model.output)

#add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

#add a final dropout rate of 0.5
x = layers.Dropout(0.5)(x)

model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = 0.0001),
              loss = 'binary_crossentropy',
              metrics = ['acc'])

history = model.fit(train_generator, 
                    validation_data = validation_generator,
                    steps_per_epoch = 100,
                    epochs = 10)

In [ ]:
history = model.fit(train_generator, 
                    validation_data = validation_generator,
                    steps_per_epoch = 100,
                    epochs = 10)

In [ ]:
#Ejecutar el entrenamiento 2 veces es igual a haberlo entrenado una sola vez pero 20Epochs